In [311]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import copy
from nltk import Tree
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer  
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [312]:
#select_type = ['NN', 'NNS', 'JJ', 'VB', 'VBD', 'VBP', 'VBZ', 'VBN', 'VBG', 'JJR', 'JJS']
select_type = ['NN', 'NNS']
letter_filter = re.compile('[^a-zA-Z]')
snowball_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
useless_word = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',\
                  'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',\
                  'z', 'as', 'an', 'and', 'are', 'were', 'at', 'by', 'down', 'for', 'from',\
                  'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out',\
                  'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through',\
                  'to', 'top', 'up', 'while', 'with'])

In [313]:
training_descrption_files = [os.path.join('data/descriptions_train', filename) \
                             for filename in os.listdir('data/descriptions_train') 
                             if filename.endswith('.txt')]
training_tag_files = [os.path.join('data/tags_train', filename)\
                      for filename in os.listdir('data/tags_train')\
                      if filename.endswith('.txt')]
test_descrption_files = [os.path.join('data/descriptions_test', filename) \
                             for filename in os.listdir('data/descriptions_test') 
                             if filename.endswith('.txt')]
training_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
training_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))

In [314]:
def remove_empty(a):
    return [i for i in a if len(i) > 0]

In [315]:

def find_type(s, type_select):
    dictionary = set()
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(s)]
    sentences = [[letter_filter.sub('', sentences[i][j].lower())\
              for j in range(len(sentences[i]))] for i in range(len(sentences))]
    sentences = [remove_empty(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    for sent in sentences:
        for e in sent:
            if e[1] in type_select:
                dictionary.add(snowball_stemmer.stem(e[0]))
    return dictionary

In [316]:
def define_des_tag_vector():
    selected = {}
    for file in training_descrption_files:
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            nvj = find_type(sent, select_type)
            for word in nvj:
                if word in selected.keys():
                    selected[word] += 1
                else:
                    selected[word] = 1
    for file in training_tag_files:
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            validTag = letter_filter.sub('', sent.split(':')[1])
            selected[validTag] = 100
    
    final = []
    for k in selected.keys():
        if selected[k] >= 2 and k not in final:
            final.append(k)
    
    final_copy = copy.deepcopy(final)
    for element in final:
        if element in useless_word:
            final_copy.remove(element)
    
    
    
    return final_copy

In [317]:
allTagFiles = [filename for filename in os.listdir('data/tags_train') if filename.endswith('.txt')]
tags = set()
for tagFile in allTagFiles:
    f = open(os.path.join('data/tags_train',tagFile), 'r+')
    for line in f:
        tag = line.split(':')[1][:-1] if line.split(':')[1].endswith('\n') else line.split(':')[1]
        tags.add(tag)
tagList = [tag for tag in tags]

In [318]:
a = define_des_tag_vector()

In [319]:
len(a)

3031

In [303]:
train_tag_files = [os.path.join('data/tags_train', filename) \
                             for filename in os.listdir('data/tags_train') 
                             if filename.endswith('.txt')]
test_tag_files = [os.path.join('data/tags_test', filename) \
                             for filename in os.listdir('data/tags_test') 
                             if filename.endswith('.txt')]
train_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))

In [304]:
tags_final = []
for tag in tagList:
    if not tag in a:
        tags_final = tags_final + tag.split()
    else:
        tags_final.append(tag)

In [305]:
def change_training_tag_weight(k):
    Y_training_1 = []
    for file in training_descrption_files:
        vector_1 = dict.fromkeys(a, 0)
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            for word in sent.split():
                i = letter_filter.sub('', word.lower())
                if i in a:
                    vector_1[i] = 1
        mm = [vector_1[element] for element in a]
        Y_training_1.append(mm)

    Y_training_2 = []
    for file in training_descrption_files:
        vector_2 = dict.fromkeys(tags_final, 0)
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            for word in sent.split():
                i = letter_filter.sub('', word.lower())
                if i in tags_final:
                    vector_2[i] = k
        mm = [vector_2[element] for element in tags_final]
        Y_training_2.append(mm)

    Y_training = []
    for i in range(10000):
        combine = Y_training_1[i] + Y_training_2[i]
        Y_training.append(combine)
    
    return Y_training

In [306]:
def change_test_tag_weight(k):
    Y_test_1 = []
    for file in test_descrption_files:
        vector_1 = dict.fromkeys(a, 0)
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            for word in sent.split():
                i = letter_filter.sub('', word.lower())
                if i in a:
                    vector_1[i] = 1
        mm = [vector_1[element] for element in a]
        Y_test_1.append(mm)

    Y_test_2 = []
    for file in test_descrption_files:
        vector_2 = dict.fromkeys(tags_final, 0)
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            for word in sent.split():
                i = letter_filter.sub('', word.lower())
                if i in tags_final:
                    vector_2[i] = k
        mm = [vector_2[element] for element in tags_final]
        Y_test_2.append(mm)


    Y_test = []
    for i in range(2000):
        combine = Y_test_1[i] + Y_test_2[i]
        Y_test.append(combine)
    return Y_test

In [307]:
Y_training = change_training_tag_weight(1)
Y_test = change_test_tag_weight(1)

In [308]:
len(Y_test)

2000

In [309]:
len(Y_test[0])

4054

In [310]:
import pickle

pickle.dump( Y_training, open( "y_train_v7.p", "wb" ) )
pickle.dump( Y_test, open( "y_test_v7.p", "wb" ) )
pickle.dump( ff, open( "alll.p", "wb" ) )

In [ ]:
patterns = """
    NP:    {<DT><WP><VBP>*<RB>*<VBN><IN><NN>}
           {<NN|NNS|NNP|NNPS><IN    >*<NN|NNS|NNP|NNPS>+}
           {<JJ>*<NN|NNS|NNP|NNPS><CC>*<NN|NNS|NNP|NNPS>+}
           {<JJ>*<NN|NNS|NNP|NNPS>+}
    """
typer = nltk.RegexpParser(patterns)